<a href="https://colab.research.google.com/github/Ariotegar/CitraKomp/blob/main/Webapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CELL 1:Install required dependencies

In [1]:
!pip install streamlit opencv-python-headless torch torchvision numpy pillow pyngrok
!pip install yolov5

# CELL 2: Configure ngrok authentication

In [2]:
from pyngrok import ngrok
ngrok.set_auth_token("API_KEY")

# CELL 3: Create the app.py file

In [7]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
import torch
from PIL import Image
import io

def extract_features(image):
    features = {}

    # Konversi PIL Image ke numpy array
    if isinstance(image, Image.Image):
        image = np.array(image)

    # Konversi ke BGR jika RGB
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # 1. Color Features
    color_hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    features['color_hist'] = color_hist.flatten()

    # 2. Texture Features
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = cv2.GaussianBlur(gray, (3, 3), 0)
    features['texture'] = np.array(glcm).flatten()

    # 3. Shape Features
    edges = cv2.Canny(gray, 100, 200)
    features['edges'] = np.sum(edges) / (edges.shape[0] * edges.shape[1])

    return features

def load_yolov5_model():
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')
    return model

def process_image(image, operation):
    # Konversi PIL Image ke numpy array
    img_array = np.array(image)

    if operation == "Grayscale":
        processed = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        return Image.fromarray(processed)

    elif operation == "Blur":
        processed = cv2.GaussianBlur(img_array, (5,5), 0)
        return Image.fromarray(processed)

    elif operation == "Edge Detection":
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        processed = cv2.Canny(gray, 100, 200)
        return Image.fromarray(processed)

    elif operation == "Sharpen":
        kernel = np.array([[-1,-1,-1],
                         [-1, 9,-1],
                         [-1,-1,-1]])
        processed = cv2.filter2D(img_array, -1, kernel)
        return Image.fromarray(processed)

    elif operation == "Feature Extraction":
        features = extract_features(img_array)
        return features

    return image

def detect_objects(image, model):
    results = model(image)
    df = results.pandas().xyxy[0]
    return results, df

def main():
    st.title("Image Processing and Object Detection App")

    # Operation
    st.sidebar.title("Operations")
    operation = st.sidebar.selectbox(
        "Select Operation",
        ["Original", "Grayscale", "Blur", "Edge Detection", "Sharpen", "Object Detection", "Feature Extraction"]
    )

    # Upload Image
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption="Original Image", use_column_width=True)

        if operation == "Feature Extraction":
            features = process_image(image, operation)

            st.subheader("Extracted Features")

            # Color Historigram
            st.write("Color Histogram Summary:")
            color_hist = features['color_hist']
            st.write(f"- Mean: {np.mean(color_hist):.2f}")
            st.write(f"- Max: {np.max(color_hist):.2f}")
            st.write(f"- Min: {np.min(color_hist):.2f}")

            # Texture Feature Summary
            st.write("\nTexture Features Summary:")
            texture = features['texture']
            st.write(f"- Mean: {np.mean(texture):.2f}")
            st.write(f"- Standard Deviation: {np.std(texture):.2f}")

            # Edge Density
            st.write("\nEdge Density:")
            st.write(f"- {features['edges']:.4f}")

        elif operation == "Object Detection":
            try:
                model = load_yolov5_model()
                results, df = detect_objects(image, model)
                results.render()
                detected_img = Image.fromarray(results.ims[0])
                st.image(detected_img, caption="Detected Objects", use_column_width=True)
                st.write("Detection Results:")
                st.dataframe(df[['name', 'confidence']].round(2))
            except Exception as e:
                st.error(f"Error in object detection: {str(e)}")

        elif operation != "Original":
            processed_image = process_image(image, operation)
            st.image(processed_image, caption=f"Processed Image ({operation})", use_column_width=True)

if __name__ == "__main__":
    main()

Overwriting app.py


# CELL 4: Upload YOLOv5 Model

In [ ]:
from google.colab import files
uploaded = files.upload()


# CELL 5: Run Streamlit with ngrok

In [8]:
!nohup streamlit run app.py &
print("Launching Streamlit app...")

import time
time.sleep(5)

try:
    public_url = ngrok.connect(8501)
    print(f"Public URL: {public_url}")
except Exception as e:
    print(f"Error: {str(e)}")

nohup: appending output to 'nohup.out'
Launching Streamlit app...
Public URL: NgrokTunnel: "https://e6dd-34-125-42-32.ngrok-free.app" -> "http://localhost:8501"


# CELL 6: Kill Process

In [6]:
# Find and kill the Streamlit process
!ps aux | grep streamlit
!pkill -9 streamlit

# Find and kill the ngrok process
!ps aux | grep ngrok
!pkill -9 ngrok

root        7891  0.0  0.0   6484  2192 ?        S    17:31   0:00 grep streamlit
root        7897  0.0  0.0   7376  3516 ?        S    17:31   0:00 /bin/bash -c ps aux | grep ngrok
root        7899  0.0  0.0   6484  2328 ?        S    17:31   0:00 grep ngrok
